In [49]:
from mnist import MNIST
from matplotlib import pyplot as plt
import numpy as np

In [50]:
from keras.datasets import mnist

In [417]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = np.reshape(train_X, (60000, 784))
test_X = np.reshape(test_X, (10000, 784))
m, n = train_X.shape
train_X = train_X.T
test_X = test_X.T
print(m)

60000


In [377]:
print(type(train_X.dtype))

<class 'numpy.dtype[float64]'>


In [52]:
print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test: ' + str(test_X.shape))
print('Y_test: ' + str(test_y.shape))


X_train: (784, 60000)
Y_train: (60000,)
X_test: (784, 10000)
Y_test: (10000,)


[1.0000001]


In [488]:
def init_params():
    W1 = np.random.rand(128, 784) - 0.5
    b1 = np.random.rand(128, 1) - 0.5
    W2 = np.random.rand(64, 128) - 0.5
    b2 = np.random.rand(64, 1) - 0.5
    W3 = np.random.rand(10, 64) - 0.5
    b3 = np.random.rand(10, 1,) - 0.5
    return  W1, b1, W2, b2, W3, b3

def Sigmoid(Z):
    return 1/(1+np.exp(-Z))

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A



def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = np.dot(W1, X) + b1
    #print(W1)
    A1 = Sigmoid(Z1)
    #print(A1.shape)
    Z2 = np.dot(W2, A1) + b2
    A2 = Sigmoid(Z2)
    Z3 = np.dot(W3, A2) + b3
    #print(A2.max())
    #print(W3.max())
    #print(np.count_nonzero(np.isnan(Z3)))
    A3 = softmax(Z3)
    #print(A3.max())
    #print(A3.min())
    #print(np.count_nonzero(np.isnan(A3)))
    #print(A3.shape)
    #print(A3)
    return(Z1, A1, Z2, A2, Z3, A3)

def Sigmoid_deriv(Z):
    dsig = (np.exp(-Z))/((np.exp(-Z)+1)**2)
    dsig = np.nan_to_num(dsig)
    #print(dsig)
    return dsig

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    #print(A3.max())
    dW3 = 1/m * dZ3.dot(A2.T)
    #print(dW3)
    db3 = 1/m * np.sum(dZ3)
    dZ2 = W3.T.dot(dZ3) * Sigmoid_deriv(Z2)
    #print(d2)
    dW2 = 1/m * dZ2.dot(A1.T)
    #print(dW2)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * Sigmoid_deriv(Z1)
    #print(Sigmoid_deriv(Z1))
    #print('Hi')
    #print(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    #print(dW1)
    #zprint(dZ1)
    db1 = 1/m * np.sum(dZ1)  
    return(dW1, db1, dW2, db2, dW3, db3)

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    #print(W1)
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3
    #print(dW1)
    #print(dW1, dW2, dW3)
    return W1, b1, W2, b2, W3, b3

In [433]:
def get_predictions(A3):
    #print(A3)
    return np.argmax(A3,0)
    

def get_accuracy(predictions, Y):
    #Clean
    print(predictions, Y)
    return np.sum(predictions == Y)/ Y.size

In [461]:
def gradient_decent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3

In [ ]:
W1, b1, W2, b2, W3, b3 = gradient_decent(train_X, train_y, 1, 500)

<ipython-input-488-9ed320554a9e>:11: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-Z))
<ipython-input-488-9ed320554a9e>:39: RuntimeWarning: overflow encountered in exp
  dsig = (np.exp(-Z))/((np.exp(-Z)+1)**2)
<ipython-input-488-9ed320554a9e>:39: RuntimeWarning: overflow encountered in square
  dsig = (np.exp(-Z))/((np.exp(-Z)+1)**2)
<ipython-input-488-9ed320554a9e>:39: RuntimeWarning: invalid value encountered in divide
  dsig = (np.exp(-Z))/((np.exp(-Z)+1)**2)


Iteration:  0
[6 0 0 ... 7 7 7] [5 0 4 ... 5 6 8]
0.06528333333333333
Iteration:  10
[3 0 1 ... 7 0 8] [5 0 4 ... 5 6 8]
0.5614166666666667
Iteration:  20
[3 0 4 ... 7 0 8] [5 0 4 ... 5 6 8]
0.7097833333333333
Iteration:  30
[3 0 4 ... 7 0 8] [5 0 4 ... 5 6 8]
0.7660166666666667
Iteration:  40
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8009666666666667
Iteration:  50
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8232333333333334
Iteration:  60
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8382833333333334
Iteration:  70
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.849
Iteration:  80
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8585333333333334
Iteration:  90
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8652166666666666
Iteration:  100
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.87175
Iteration:  110
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8775666666666667
Iteration:  120
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8807166666666667
Iteration:  130
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8854
Iteration:  140
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]


NameError: name 'Z3' is not defined